# TensorFlow データセット

TFDS は、TensorFlow、Jax、およびその他の機械学習フレームワークで使用するための、すぐに使用できるデータセットのコレクションを提供します。

データのダウンロードと準備を決定論的に処理し、 `tf.data.Dataset` (または`np.array` ) を構築します。

注: [TFDS](https://www.tensorflow.org/datasets) (このライブラリ) と`tf.data` (効率的なデータ パイプラインを構築するための TensorFlow API) を混同しないでください。 TFDS は、 `tf.data`の高レベルのラッパーです。この API に慣れていない場合は、まず[公式の tf.data ガイド](https://www.tensorflow.org/guide/data)を読むことをお勧めします。


Copyright 2018 The TensorFlow Datasets Authors、Apache License、バージョン 2.0 に基づいてライセンス供与

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/datasets/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で見る</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行する</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/datasets/blob/master/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/datasets/docs/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
</td>
</table>

## インストール

TFDS は 2 つのパッケージで存在します。

- `pip install tensorflow-datasets` : 数か月ごとにリリースされる安定バージョン。
- `pip install tfds-nightly` : 毎日リリースされ、データセットの最新バージョンが含まれます。

この colab は`tfds-nightly`を使用します。


In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## 利用可能なデータセットを見つける

すべてのデータセット ビルダーは`tfds.core.DatasetBuilder`のサブクラスです。利用可能なビルダーのリストを取得するには、 `tfds.list_builders()`を使用するか、[カタログ](https://www.tensorflow.org/datasets/catalog/overview)を参照してください。

In [ ]:
tfds.list_builders()

## データセットをロードする

### tfds.load

データセットをロードする最も簡単な方法は`tfds.load`です。それは：

1. データをダウンロードし、 [`tfrecord`](https://www.tensorflow.org/tutorials/load_data/tfrecord)ファイルとして保存します。
2. `tfrecord`をロードし、 `tf.data.Dataset`を作成します。


In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

いくつかの一般的な議論:

- `split=` : どの分割を読み取るか (例: `'train'` 、 `['train', 'test']` 、 `'train[80%:]'` 、...)。[分割 API ガイドを](https://www.tensorflow.org/datasets/splits)参照してください。
- `shuffle_files=` : 各エポック間でファイルをシャッフルするかどうかを制御します (TFDS は大きなデータセットを複数の小さなファイルに保存します)。
- `data_dir=` : データセットが保存される場所 (デフォルトは`~/tensorflow_datasets/` )
- `with_info=True` : データセットのメタデータを含む`tfds.core.DatasetInfo`を返します。
- `download=False` : ダウンロードを無効にします


### tfds.builder

`tfds.load` 、 `tfds.core.DatasetBuilder`の薄いラッパーです。 `tfds.core.DatasetBuilder` API を使用して同じ出力を取得できます。

In [ ]:
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

### `tfds build` CLI

特定のデータセットを生成したい場合は、[`tfds`コマンド ライン](https://www.tensorflow.org/datasets/cli)を使用できます。例えば：

```sh
tfds build mnist
```

利用可能なフラグについては[ドキュメントを](https://www.tensorflow.org/datasets/cli)参照してください。

## データセットを反復処理する

### 辞書として

デフォルトでは、 `tf.data.Dataset`オブジェクトには`tf.Tensor`の`dict`含まれています。

In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

`dict`キーの名前と構造を確認するには、[カタログ](https://www.tensorflow.org/datasets/catalog/overview#all_datasets)内のデータセットのドキュメントを参照してください。例: [mnist ドキュメント](https://www.tensorflow.org/datasets/catalog/mnist)。

### タプルとして ( `as_supervised=True` )

`as_supervised=True`を使用すると、教師ありデータセットの代わりにタプル`(features, label)`を取得できます。

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in ds:  # example is (image, label)
  print(image.shape, label)

### numpy として ( `tfds.as_numpy` )

`tfds.as_numpy`を使用して以下を変換します。

- `tf.Tensor` -&gt; `np.array`
- `tf.data.Dataset` -&gt; `Iterator[Tree[np.array]]` ( `Tree`任意のネストされた`Dict` 、 `Tuple`にすることができます)


In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in tfds.as_numpy(ds):
  print(type(image), type(label), label)

### バッチ化された tf.Tensor ( `batch_size=-1` ) として

`batch_size=-1`を使用すると、単一のバッチで完全なデータセットを読み込むことができます。

This can be combined with `as_supervised=True` and `tfds.as_numpy` to get the the data as `(np.array, np.array)`:


In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print(type(image), image.shape)

データセットがメモリに収まることと、すべての例が同じ形状であることに注意してください。

## データセットのベンチマークを行う

データセットのベンチマークは、任意の反復可能オブジェクト (例: `tf.data.Dataset` 、 `tfds.as_numpy`など) に対する単純な`tfds.benchmark`呼び出しです。


In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.batch(32).prefetch(1)

tfds.benchmark(ds, batch_size=32)
tfds.benchmark(ds, batch_size=32)  # Second epoch much faster due to auto-caching

- バッチ サイズごとに結果を正規化することを忘れないでください`batch_size=` kwarg を使用します。
- 要約すると、最初のウォームアップ バッチは、 `tf.data.Dataset`追加のセットアップ時間 (バッファの初期化など) をキャプチャするために他のバッチから分離されています。
- [TFDS 自動キャッシュ](https://www.tensorflow.org/datasets/performances#auto-caching)のおかげで、2 回目の反復がどのように高速になっているかに注目してください。
- `tfds.benchmark`さらなる分析のために検査できる`tfds.core.BenchmarkResult`を返します。

### エンドツーエンドのパイプラインを構築する

さらに詳しく見ると、次のようになります。

- 完全なトレーニング パイプライン (バッチ処理、シャッフルなど) を確認するための[エンドツーエンドの Keras の例](https://www.tensorflow.org/datasets/keras_example)。
- パイプラインの速度を向上させるための[パフォーマンス ガイド](https://www.tensorflow.org/datasets/performances)(ヒント: データセットのベンチマークには`tfds.benchmark(ds)`を使用します)。


## 視覚化

### tfds.as_dataframe

`tf.data.Dataset`オブジェクトは、 <code>tfds.as_dataframe</code>を使用して<a><code data-md-type="codespan">pandas.DataFrame</code></a>に変換し、 [Colab](https://colab.research.google.com)で視覚化できます。

- 画像、音声、テキスト、ビデオなどを視覚化するには、 `tfds.as_dataframe`の 2 番目の引数として`tfds.core.DatasetInfo`を追加します。
- 最初の`x`個の例のみを表示するには、 `ds.take(x)`を使用します。 `pandas.DataFrame`完全なデータセットをメモリ内にロードするため、表示に非常にコストがかかる可能性があります。

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4), info)

### tfds.show_examples

`tfds.show_examples` `matplotlib.figure.Figure`を返します (現在サポートされているのは画像データセットのみです)。

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

fig = tfds.show_examples(ds, info)

## データセットのメタデータにアクセスする

すべてのビルダーには、データセットのメタデータを含む`tfds.core.DatasetInfo`オブジェクトが含まれています。

次の方法でアクセスできます。

- `tfds.load` API:


In [ ]:
ds, info = tfds.load('mnist', with_info=True)

- `tfds.core.DatasetBuilder` API:

In [ ]:
builder = tfds.builder('mnist')
info = builder.info

データセット情報には、データセットに関する追加情報 (バージョン、引用、ホームページ、説明など) が含まれています。

In [ ]:
print(info)

### 機能のメタデータ (ラベル名、画像の形状など)

`tfds.features.FeatureDict`にアクセスします。

In [ ]:
info.features

クラス数、ラベル名:

In [ ]:
print(info.features["label"].num_classes)
print(info.features["label"].names)
print(info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
print(info.features["label"].str2int('7'))

形状、dtype:

In [ ]:
print(info.features.shape)
print(info.features.dtype)
print(info.features['image'].shape)
print(info.features['image'].dtype)

### 分割メタデータ (分割名、例の数など)

`tfds.core.SplitDict`にアクセスします。

In [ ]:
print(info.splits)

利用可能な分割:

In [ ]:
print(list(info.splits.keys()))

個々の分割に関する情報を取得します。

In [ ]:
print(info.splits['train'].num_examples)
print(info.splits['train'].filenames)
print(info.splits['train'].num_shards)

サブスプリット API でも動作します。

In [ ]:
print(info.splits['train[15%:75%]'].num_examples)
print(info.splits['train[15%:75%]'].file_instructions)

## トラブルシューティング

### 手動ダウンロード (ダウンロードが失敗した場合)

何らかの理由（オフラインなど）でダウンロードが失敗した場合。データを自分でいつでも手動でダウンロードして、 `manual_dir` (デフォルトは`~/tensorflow_datasets/downloads/manual/`に配置できます。

ダウンロードする URL を確認するには、次の点を調べてください。

- 新しいデータセットの場合 (フォルダーとして実装): [`tensorflow_datasets/`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/) `<type>/<dataset_name>/checksums.tsv` 。例: [`tensorflow_datasets/datasets/bool_q/checksums.tsv`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/datasets/bool_q/checksums.tsv) 。

    データセットのソースの場所は[カタログ](https://www.tensorflow.org/datasets/catalog/overview)で見つけることができます。

- 古いデータセットの場合: [`tensorflow_datasets/url_checksums/<dataset_name>.txt`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/url_checksums)

### `NonMatchingChecksumError`の修正

TFDS は、ダウンロードされた URL のチェックサムを検証することで決定性を保証します。 `NonMatchingChecksumError`が発生した場合は、次のことを示している可能性があります。

- Web サイトがダウンしている可能性があります (例`503 status code` )。 URLをご確認ください。
- Google ドライブの URL の場合は、同じ URL にアクセスする人が多すぎるとドライブがダウンロードを拒否することがあるため、後でもう一度試してください。[バグを](https://github.com/tensorflow/datasets/issues/1482)参照
- 元のデータセット ファイルが更新されている可能性があります。この場合、TFDS データセット ビルダーを更新する必要があります。新しい Github の問題または PR を開いてください:
    - `tfds build --register_checksums`を使用して新しいチェックサムを登録します。
    - 最終的にはデータセット生成コードを更新します。
    - データセットの`VERSION`を更新します
    - データセットを更新します`RELEASE_NOTES` : チェックサムが変更された原因は何ですか?いくつかの例は変更されましたか?
    - データセットがまだ構築できることを確認してください。
    - PRを送信してください

注: `~/tensorflow_datasets/download/`でダウンロードしたファイルを検査することもできます。


## 引用

論文に`tensorflow-datasets`使用している場合は、使用したデータセット ([データセット カタログ](https://www.tensorflow.org/datasets/catalog/overview)で見つけることができます) に固有の引用に加えて、次の引用を含めてください。

```
@misc{TFDS,
  title = { {TensorFlow Datasets}, A collection of ready-to-use datasets},
  howpublished = {\url{https://www.tensorflow.org/datasets}},
}
```